In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# experiment 1:
# first, creating a set of N randomly generated (m, 1) vectors:
import numpy as np
m = 10
N = 1000
# vectors = [np.random.rand(*(m, 1)) for _ in range(N)] # uniform distribution between 0 and 1
vectors = [np.random.randn(*(m, 1)) for _ in range(N)] # normal dist so can get negative vals too
# print (vectors)

In [3]:
# finding nearest neighbor of each vector using O(N^2) brute force approach:
'''
nearest_neighbors = {} # the keys will be the indices of the vectors from 0 to N-1, and the corresponding values will be the indices of its nearest neighbor
for i in range(len(vectors)):
    nearest_dist = float('inf')
    nearest_neighbors[i] = None
    for j in range(len(vectors)):
        if j != i:
            dist = np.linalg.norm(vectors[i] - vectors[j])
            if dist < nearest_dist:
                nearest_dist = dist
                nearest_neighbors[i] = j
print (nearest_neighbors) 
'''
# more efficient approach (although technically same big O runtime complexiity i think cuz N^2 vs N(N+1)/2) which will also allow us to find top 5 nearest neighbors easily
pairwise_distances = {} # {0: {1: 0.1, 2: 0.3, 3: 0.6}, 1: {0: 0.1, 2: 0.7, 3: 0.8}}
for i in range(len(vectors)):
    for j in range(i+1, len(vectors)):
        dist = np.linalg.norm(vectors[i] - vectors[j])
        #pairwise_distances[(i, j)] = dist
        try:
            pairwise_distances[i][j] = dist
        except:
            pairwise_distances[i] = {j: dist}
        try:
            pairwise_distances[j][i] = dist
        except:
            pairwise_distances[j] = {i: dist}
nearest_neighbors = {}
five_nearest_neighbors = {}
for i in list(pairwise_distances.keys()):
    nearest_neighbors[i] = min(pairwise_distances[i], key=pairwise_distances[i].get)
    five_nearest_neighbors[i] = list(dict(sorted(pairwise_distances[i].items(), key=lambda item: item[1])).keys())[:5]
print (nearest_neighbors)
print (five_nearest_neighbors)

{0: 466, 1: 916, 2: 775, 3: 869, 4: 891, 5: 414, 6: 441, 7: 757, 8: 910, 9: 339, 10: 830, 11: 109, 12: 848, 13: 474, 14: 415, 15: 695, 16: 686, 17: 75, 18: 374, 19: 491, 20: 391, 21: 900, 22: 898, 23: 57, 24: 242, 25: 448, 26: 700, 27: 629, 28: 821, 29: 277, 30: 35, 31: 530, 32: 916, 33: 338, 34: 334, 35: 715, 36: 550, 37: 768, 38: 795, 39: 562, 40: 272, 41: 667, 42: 455, 43: 736, 44: 255, 45: 206, 46: 196, 47: 649, 48: 395, 49: 584, 50: 11, 51: 526, 52: 769, 53: 867, 54: 885, 55: 562, 56: 430, 57: 668, 58: 191, 59: 935, 60: 246, 61: 277, 62: 59, 63: 232, 64: 678, 65: 728, 66: 401, 67: 613, 68: 231, 69: 384, 70: 361, 71: 924, 72: 898, 73: 990, 74: 742, 75: 930, 76: 901, 77: 497, 78: 195, 79: 319, 80: 38, 81: 971, 82: 490, 83: 724, 84: 812, 85: 177, 86: 812, 87: 573, 88: 302, 89: 249, 90: 101, 91: 557, 92: 51, 93: 965, 94: 240, 95: 568, 96: 413, 97: 695, 98: 822, 99: 659, 100: 789, 101: 500, 102: 185, 103: 418, 104: 562, 105: 163, 106: 747, 107: 564, 108: 741, 109: 490, 110: 913, 111: 2

In [4]:
# finding nearest neighbor of each vector using NaiveLSH:
from memristor.engine.model import NaiveLSH
from memristor.crossbar.model import LineResistanceCrossbar
from memristor.devices import StaticMemristor
# naive_lsh = NaiveLSH(
#     hash_size=10, # adjustable hyperparameter
#     crossbar_class=LineResistanceCrossbar,
#     crossbar_params={'r_wl': 20, 'r_bl': 20, 'r_in':10, 'r_out':10, 'V_SOURCE_MODE':'|_|'},
#     memristor_model_class=StaticMemristor,
#     memristor_params={'frequency': 1e8, 'temperature': 273 + 40},
#     m=m,
#     r=1, # adjustable hyperparameter
# )
reps = 3 # adjustable hyperparameter (repetitions of the hashing)
all_bins = []
for _ in range(reps):
    naive_lsh = NaiveLSH(
        hash_size=5, # adjustable hyperparameter
        crossbar_class=LineResistanceCrossbar,
        crossbar_params={'r_wl': 20, 'r_bl': 20, 'r_in':10, 'r_out':10, 'V_SOURCE_MODE':'|_|'},
        memristor_model_class=StaticMemristor,
        memristor_params={'frequency': 1e8, 'temperature': 273 + 40},
        m=m,
        r=1, # adjustable hyperparameter
    )
    bins = {}
    for i in range(len(vectors)):
        hash = naive_lsh.inference(vectors[i])
        # print (hash)
        if hash not in bins.keys():
            bins[hash] = [i]
        else:
            bins[hash].append(i)
    for bin in list(bins.values()):
        all_bins.append(bin)
#      {010:[1,5,7], 111:[5,6,7]}

# now at this point all_bins is a list like [[1,2], [1,3,5], ... ] where each element of all_bins is a bin containing indices of vectors that are likely
# to be close to each other. so now to find the nearest neighbor for each vector, we simply iterate through and check only those vectors that share a bin
# with it, so in this case for 1 we would check 2, 3, and 5, to find the nearest neighbor
nearest_neighbors_approx = {}
for i in range(len(vectors)):
    nearest_dist = float('inf')
    nearest_neighbors_approx[i] = None
    for bin in all_bins:
        if len(bin) > N/reps:
            continue
        if i in bin:
            for j in bin:
                if j != i:
                    dist = np.linalg.norm(vectors[i] - vectors[j])
                    if dist < nearest_dist:
                        nearest_dist = dist 
                        nearest_neighbors_approx[i] = j
print (nearest_neighbors_approx)

{0: 466, 1: 916, 2: 775, 3: 869, 4: 891, 5: 414, 6: 441, 7: 757, 8: 910, 9: 339, 10: 830, 11: 109, 12: 848, 13: 474, 14: 415, 15: 695, 16: 606, 17: 75, 18: 374, 19: 127, 20: 391, 21: 900, 22: 898, 23: 57, 24: 242, 25: 571, 26: 700, 27: 629, 28: 821, 29: 371, 30: 35, 31: 530, 32: 362, 33: 338, 34: 334, 35: 715, 36: 550, 37: 115, 38: 795, 39: 460, 40: 272, 41: 667, 42: 455, 43: 736, 44: 255, 45: 426, 46: 196, 47: 649, 48: 395, 49: 584, 50: 11, 51: 526, 52: 769, 53: 867, 54: 885, 55: 858, 56: 430, 57: 668, 58: 325, 59: 935, 60: 246, 61: 277, 62: 59, 63: 404, 64: 678, 65: 728, 66: 401, 67: 613, 68: 231, 69: 384, 70: 361, 71: 345, 72: 898, 73: 990, 74: 742, 75: 930, 76: 901, 77: 497, 78: 195, 79: 422, 80: 997, 81: 971, 82: 156, 83: 724, 84: 812, 85: 177, 86: 812, 87: 573, 88: 302, 89: 271, 90: 101, 91: 557, 92: 356, 93: 723, 94: 19, 95: 568, 96: 413, 97: 695, 98: 822, 99: 659, 100: 789, 101: 500, 102: 5, 103: 418, 104: 562, 105: 163, 106: 747, 107: 564, 108: 741, 109: 490, 110: 913, 111: 23

In [5]:
print (nearest_neighbors)
print (nearest_neighbors_approx)

{0: 466, 1: 916, 2: 775, 3: 869, 4: 891, 5: 414, 6: 441, 7: 757, 8: 910, 9: 339, 10: 830, 11: 109, 12: 848, 13: 474, 14: 415, 15: 695, 16: 686, 17: 75, 18: 374, 19: 491, 20: 391, 21: 900, 22: 898, 23: 57, 24: 242, 25: 448, 26: 700, 27: 629, 28: 821, 29: 277, 30: 35, 31: 530, 32: 916, 33: 338, 34: 334, 35: 715, 36: 550, 37: 768, 38: 795, 39: 562, 40: 272, 41: 667, 42: 455, 43: 736, 44: 255, 45: 206, 46: 196, 47: 649, 48: 395, 49: 584, 50: 11, 51: 526, 52: 769, 53: 867, 54: 885, 55: 562, 56: 430, 57: 668, 58: 191, 59: 935, 60: 246, 61: 277, 62: 59, 63: 232, 64: 678, 65: 728, 66: 401, 67: 613, 68: 231, 69: 384, 70: 361, 71: 924, 72: 898, 73: 990, 74: 742, 75: 930, 76: 901, 77: 497, 78: 195, 79: 319, 80: 38, 81: 971, 82: 490, 83: 724, 84: 812, 85: 177, 86: 812, 87: 573, 88: 302, 89: 249, 90: 101, 91: 557, 92: 51, 93: 965, 94: 240, 95: 568, 96: 413, 97: 695, 98: 822, 99: 659, 100: 789, 101: 500, 102: 185, 103: 418, 104: 562, 105: 163, 106: 747, 107: 564, 108: 741, 109: 490, 110: 913, 111: 2

In [6]:
print (nearest_neighbors == nearest_neighbors_approx)
count = 0
cnt = 0
for i in range(N):
    if nearest_neighbors_approx[i] == nearest_neighbors[i]:
        count += 1
    if nearest_neighbors_approx[i] in five_nearest_neighbors[i]:
        cnt += 1
accuracy = count/N
top5_accuracy = cnt/N
print (accuracy)
print (top5_accuracy)

False
0.797
0.989


In [7]:
print (len(all_bins))
print (all_bins)
print ([len(bin) for bin in all_bins])
print ([len(bin) for bin in all_bins if len(bin) <= N/3])

66
[[0, 5, 7, 18, 22, 41, 43, 46, 50, 56, 72, 76, 78, 79, 84, 87, 100, 102, 116, 121, 128, 140, 151, 153, 169, 172, 173, 178, 190, 193, 196, 199, 220, 228, 230, 233, 243, 244, 247, 256, 275, 278, 281, 299, 300, 306, 311, 317, 323, 332, 333, 348, 353, 367, 371, 374, 383, 394, 400, 403, 414, 422, 440, 447, 449, 453, 467, 470, 478, 481, 484, 499, 501, 508, 514, 522, 527, 534, 536, 583, 589, 597, 598, 600, 602, 633, 636, 640, 641, 644, 653, 655, 671, 672, 673, 681, 684, 691, 730, 736, 737, 744, 749, 753, 764, 767, 784, 789, 796, 805, 812, 822, 831, 847, 852, 860, 865, 871, 879, 880, 887, 898, 901, 917, 938, 946, 947, 954, 962, 965, 970, 975, 983, 992, 995, 998], [1, 27, 30, 35, 104, 133, 179, 213, 227, 464, 530, 554, 555, 562, 645, 706, 759, 793, 858], [2, 15, 16, 23, 34, 39, 54, 60, 68, 82, 83, 89, 97, 99, 115, 126, 138, 152, 156, 176, 191, 194, 201, 202, 210, 231, 241, 246, 248, 253, 259, 263, 271, 279, 313, 327, 334, 352, 361, 385, 398, 405, 410, 429, 444, 455, 504, 510, 533, 541, 542, 

In [8]:
# when experimenting with like a bigger dataset and stuff come up with  a metric to compare these 2 dicts
# also compare the runtime complexities, cuz its possible that its working so well because of sth wrong in the implementation whiich results in the runtime
# just being the same as the brute force method

In [9]:
'''
things to check: 
- sizes of the bins, should not be big because otherwise there would be no runtime improvement -> get metrics for reduction of search space/runtime or space complexity
- varying the parameters like N, m, hash size, etc. (try bigger/more data)
- using lineres_memristive_vmm not naive_memristive_vmm
- experiment with varying non-idealities
- experiment 2, create visualizations
- write section 4 of the paper    (rn highest priority is finalizing the experiment and section 4 of the paper)
- is the change made in StaticMemristor fine?
'''

'\nthings to check: \n- sizes of the bins, should not be big because otherwise there would be no runtime improvement -> get metrics for reduction of search space/runtime or space complexity\n- varying the parameters like N, m, hash size, etc. (try bigger/more data)\n- using lineres_memristive_vmm not naive_memristive_vmm\n'